In [ ]:
""" 
Setting up analysis for model comparison, where each trial/task is a datapoiont
(differs from strok model where each action is datapoint)
- This is mostly setup, but some plotting analysis code.
- Mostly has been moved to analysis in other notebooks.
[OBSOLETE] I think (7/11/21)
"""

In [ ]:
%load_ext autoreload
%autoreload 2

%cd ..

In [ ]:
from tools.utils import * 
from tools.plots import *
from tools.analy import *
from tools.calc import *
from tools.analyplot import *
from tools.preprocess import *
from tools.dayanalysis import *

from pythonlib.drawmodel.analysis import *
from pythonlib.tools.stroketools import *
from pythonlib.drawmodel.strokedists import *
from pythonlib.analysis.probedatTaskmodel import *

In [ ]:
animal = "Red"
expt = "lines5"
# date = 200924
date = 200929
session = 1

fd = loadSingleData(animal, date, expt, session, resave_as_dict=False, load_resaved_data=True, 
                          resave_overwrite=False)




### helper to partition trials into train and test

In [ ]:
# 1) Load filedata across days
from analysis.modelexpt import loadMultDataForExpt
from analysis.modelexpt import *
    
def _loadanimal(a):

    expt="lines5"
    FD, MD = loadMultDataForExpt(expt, a, "all")

    # 2) Convert to probedat
    PROBEDAT = loadProbeDatWrapper(FD, MD)
    
    return PROBEDAT

In [ ]:
def splitter(probedat, params):
    """ takes a entire PROBEDAT list and outputs list of ["train", "test", ...]
    """
    # Random split
    from sklearn.model_selection import train_test_split
    probedat_train, probedat_test = train_test_split(probedat, test_size=params["test_size"])
    
    return probedat_train, probedat_test

# OBSOLETE THIS IS IN PROBEDATTASK CLASSS
# def pd2strokes(probedat):
#     strokes, tasks, fix, strokes_task = [[], [], [], []]
#     for p in probedat:
#         fd = p["filedata"]
#         t = p["trial"]
#         S = getTrialsStrokesByPeanuts(fd, t)
#         if len(S)>0:
#             strokes.append(S)
#             tasks.append(getTrialsTask(fd, t))
#             fix.append(getTrialsFix(fd, t)["fixpos_pixels"])
#             strokes_task.append(getTrialsTaskAsStrokes(fd, t))
#     for s, t, f in zip(strokes_task, tasks, fix):
#         t["strokes"] = s
#         t["fixpos"] = f
#     return strokes, tasks

        
from pythonlib.drawmodel.analysis import *

# OBSOLETE -T HIS IS IN THE PROBEDATTASK CALSS
# def _make(probedat_train, priorver="distance_travel", parse_ver="permutations", 
#          chunkmodel = None, name="test", posterior_ver="weighted"):
    
#     # ---1) Build model
#     PARAMS = {
#         "getPriors":{"normscore":[0.01, True]},
#         "getLikelis":None,
#         "getPosteriors":None,
#         "parse":{"parses_split":False, "parses_bothdir":False},
#         "posterior_ver":posterior_ver,
#         "prior_norm_ver":"softmax",
#         "standardize":{"standardize_strokes":False},
#         "run_only_once":['parse', 'prior_score', 'getLikelis']
#     }

#     # 3) Prepare model
#     PARAMS_MODEL = {
#         "use_presaved_parses":False,
#         "priorver":priorver,
#         "likeliver":"segments",
#         "parse_ver":parse_ver,
#         "chunkmodel":chunkmodel,
#         "name":name
#     }

#     strokes, tasks = pd2strokes(probedat_train)
#     mod = Model(PARAMS_MODEL)
#     data = Dataset(strokes, tasks, PARAMS=PARAMS)
#     data.applyModel(mod)
    
#     return data




In [ ]:
# OBSOLETE - this is in taskmodel.py
def parsefun(task, threshdist="adaptive", returnkeeplist=False):
    """ get all partitions that have at least one
    chunk of 2 strokes or more, and no chunks of
    3 strokes or more.
    - only chunks strokes if they are close"""
    from pythonlib.drawmodel.strokedists import distmatStrokes    
    
    # -- get stroke indices
    strokes_task = convertTask2Strokes(task)
    tmp = [i for i in range(len(strokes_task))]
    
    # -- threshdist make mean of stroke lengths
    if threshdist=="adaptive":
        td = 0.75*np.mean(strokeDistances(strokes_task))
    else:
        td = threshdist

    # -- get all partitions.
    from pythonlib.tools.listtools import partition
    A = list(partition(tmp))

    # -- keep only partitions that have at least one multi-stroke chunk
    A = [a for a in A if len(a)<len(tmp)]
    # -- dont keep if have a chunk with >2 strokes chunked
    def tmp(x):
        return [len(xx)>2 for xx in x]
    A = [a for a in A if not any(tmp(a))]
    
    # -- check each chunk proximitiy - only keep if close.
    Anew = []
    keeplist = []
    for a in A:
        keep = True
        for chunk in a:
            if len(chunk)>1:
                # check if strokes in this chunk are close
                # for all pairs of strok, compute min dist.
                # throw out chunk if any pair has dist greater than some thresh
                strokes1 = [strokes_task[n] for n in chunk]
                d = distscalarStrokes(strokes1, strokes1, ver="mindist_offdiag")
                if d>td:
                    keep=False
#                     f, ax = plt.subplots()
#                     plotDatStrokes(strokes1, ax)
#                     assert False

        if keep:
            Anew.append(a)
        keeplist.append(keep)
    A = Anew
    
    if returnkeeplist:
        return A, keeplist
    else:
        return A


allkeeps =[]
for P in probedat:
    task = getTrialsTask(P["filedata"], P["trial"])
    allkeeps.extend(parsefun(task, threshdist="adaptive", returnkeeplist=True)[1])
print("frac chunks kept after thresholding")
print(sum(allkeeps)/len(allkeeps))

## EXPERIMENT

In [ ]:
# try idfferent models (train test split) on same datset. 
# This is using old version before made  ProbedatTaskmodel class.
# Should update this to use that class. should be easey.
# each model/data run should be one class instance, need to update
# to transfer params to test on test tasks...


In [ ]:
out = []
Nsplits = 20
from pythonlib.tools.dicttools import filterDict
from pythonlib.tools.modfittools import minimize

for animal in ["Pancho", "Red"]:
    
    PROBEDAT = _loadanimal(animal)
    
    for epoch in [1,2]:
        partition = {
            "datagetter":{
                "taskgroup":"train_fixed",
                "epoch":epoch,
                "insummarydates":True
            },
            "splitter":splitter,
            "splitparams":{
                "test_size":0.1
            }
        }

        # 1) get data pool
        probedat = filterDict(PROBEDAT, partition["datagetter"])

        for n in range(Nsplits):
            probedat_train, probedat_test = splitter(probedat, partition["splitparams"])

            for priorver, parse_ver, chunkmodel, name in zip(
                ["distance_travel", "uniform", "distance_travel", "distance_travel","uniform","uniform"], 
                ["permutations", "permutations", "chunks", "chunks", "chunks", "chunks"],
                [None, None, "3line", parsefun, "3line", parsefun],
                ["distance_travel", "null", "3line-dist", "linePlusL-dist", "3line", "linePlusL"]):

                # 1) make dataset/model
                data = _make(probedat_train, priorver, parse_ver, chunkmodel, name)

                # 2) optimize
                params0 = [0.1]
                bounds = [(0,1)]
                def fun(params):
                    data.PARAMS["getPriors"]["normscore"][0] = params[0]
                    data.run()
                    return -np.mean(data.summarizeScore()["post_scores"])
                minimize(fun, params0, bounds)

                # 3) apply to test set
                strokes, tasks = pd2strokes(probedat_test)
                datatest = transferParams(data, strokes, tasks)

                # 4) collect outputs
                out.append({
                    "animal":animal,
                    "epoch":epoch,
                    "splitn":n,
                    "priorver":priorver,
                    "parse_ver":parse_ver, 
                    "chunkmodel":chunkmodel,
                    "name":name,
                    "postscores_train":data.summarizeScore()["post_scores"],
                    "postscores_test":datatest.summarizeScore()["post_scores"],
                    "PARAMS_fit": data.PARAMS["getPriors"]["normscore"][0]
                })



In [ ]:
for o in out:
    o["score_train"] = np.mean(o["postscores_train"])
    o["score_test"] = np.mean(o["postscores_test"])
    

In [ ]:
## ======= SAVE
from pythonlib.tools.expttools import makeTimeStamp

expt ="lines5"
SAVEDIR = f"{PROBEDAT[0]['filedata']['params']['figuredir_notebook']}/analysis_setuptaskmodel/{expt}/{makeTimeStamp('modelcomp', False)}"
import os
os.makedirs(SAVEDIR, exist_ok=True)





In [ ]:
import pandas as pd
import seaborn as sns
OUT = pd.DataFrame(out)

# OUT
fig = sns.catplot(data=OUT, x="name", row="animal", y="score_train", hue="epoch", aspect=3)
fig.savefig(f"{SAVEDIR}/scores_train.pdf")
fig = sns.catplot(data=OUT, x="name", row="animal", y="score_test", hue="epoch", aspect=3)
fig.savefig(f"{SAVEDIR}/scores_test.pdf")

In [ ]:
fig = sns.catplot(data=OUT, x="name", row="animal", y="PARAMS_fit", hue="epoch", aspect=3)
fig.savefig(f"{SAVEDIR}/params_fit.pdf")

In [ ]:
## = SAVE
import pickle
with open(f"{SAVEDIR}/out.pkl", "wb") as f:
    pickle.dump(out, f)

## Build multiple models - score each task by comparing

In [ ]:
## OLSOLETE - this is all folded in to probedatOfflineScore

In [ ]:
# expt="lines5"
# for animal in ["Pancho", "Red"]:    
#     PROBEDAT = _loadanimal(animal)

#     SAVEDIR = f"{PROBEDAT[0]['filedata']['params']['figuredir_notebook']}/analysis_setuptaskmodel/{expt}/{makeTimeStamp('modelcomp_rescoring', False)}"
#     import os
#     os.makedirs(SAVEDIR, exist_ok=True)
#     for taskgroup in ["train_fixed", "G2", "G3"]:
#         partition = {
#             "datagetter":{
#                 "taskgroup":taskgroup,
#                 "insummarydates":True
#             },
#             "splitter":splitter,
#             "splitparams":{
#                 "test_size":0.1
#             }
#         }

#         # 1) get data pool
#         probedat = filterDict(PROBEDAT, partition["datagetter"])
#         DATAS = {}
        
#         for priorver, parse_ver, chunkmodel, posterior_ver, name in zip(
#             ["distance_travel","uniform","uniform"], 
#             ["permutations", "chunks", "chunks"],
#             [None, "3line", parsefun],
#             ["weighted", "maxlikeli", "maxlikeli"],
#             ["distance_travel", "3line", "linePlusL"]):

#             # 1) make dataset/model
#             data = _make(probedat, priorver, parse_ver, chunkmodel, name, posterior_ver=posterior_ver)

#             # 2) optimize
#             params0 = [0.1]
#             bounds = [(0,1)]
#             def fun(params):
#                 data.PARAMS["getPriors"]["normscore"][0] = params[0]
#                 data.run()
#                 return -np.mean(data.summarizeScore()["post_scores"])
#             minimize(fun, params0, bounds)

#             # save
#             DATAS[name] = data
        
#         for D in DATAS.values():
#             assert len(D.trials)==len(probedat)
            
#         ## reassign model scores 
#         modelscores = []
#         for i, p in enumerate(probedat):
#             modelscores.append({})
#             modelscores[-1]["online_modelcomp"] = p["modelcomp"]
#             modelscores[-1]["epoch"] = p["epoch"]
#             for name, data in DATAS.items():
#                 modelscores[-1][name] = data.trials[i]["posterior"]
#             modelscores[-1]["offline_modelcomp"] = 2*(modelscores[-1]["3line"]/(modelscores[-1]["3line"] + modelscores[-1]["linePlusL"])-0.5)

#         MS = pd.DataFrame(modelscores)
#         fig = sns.pairplot(data=MS, vars = ["online_modelcomp", "offline_modelcomp", "distance_travel", "3line", "linePlusL"], hue="epoch")
#         fig.savefig(f"{SAVEDIR}/overview_pairplot-{animal}-{taskgroup}.pdf")

In [ ]:
len([p for p in probedat if len(getTrialsStrokesByPeanuts(p["filedata"], p["trial"]))>0])

In [ ]:
        MS = pd.DataFrame(modelscores)
        fig = sns.pairplot(data=MS, vars = ["online_modelcomp", "offline_modelcomp", "distance_travel", "3line", "linePlusL"], hue="epoch")
        fig.savefig(f"{SAVEDIR}/overview_pairplot-{animal}-{taskgroup}.pdf")

In [ ]:
def stroke2features(strokes, features=["center"]):
    """ convert strokes (list of rray)
    to list of scalar values for features
    of interest
    """
    featdict = []
    for s in strokes:
        featdict = 
        for f in features:
            if f=="center":
                

## Reassigning scores to probedat
#### also trying new class

In [ ]:
from analysis.probedatTaskmodel import *

In [ ]:
PROBEDAT = _loadanimal("Red")

In [ ]:
## [IMPORTANT] This is function tthat upadtes probedat correctly.
probedat, fig = probedatOfflineScore(probedat, filtdict={"random_task":[False]},
                        ploton=True)


### Visualize tasks, sorted by model scores

In [ ]:
# Extract model scores.
probedat, PDdict = probedatOfflineScore(probedat, filtdict={"random_task":[False]},
                        ploton=True, return_all_models=True)


In [ ]:
t = 2
for name, P in PDdict.items():
    P.Datamodel.plotExampleTrial(t)

In [ ]:
likelis = [p["likeli"] for p in PDdict["distance_travel"].Datamodel.trials[1]["model_parses"]]

np.argmax(likelis)

In [ ]:
from pythonlib.tools.expttools import makeTimeStamp
expt ="lines5"
SAVEDIR = f"{PROBEDAT[0]['filedata']['params']['figuredir_notebook']}/analysis_setuptaskmodel/{expt}/{makeTimeStamp('modelcomp', False)}"
import os
os.makedirs(SAVEDIR, exist_ok=True)


In [ ]:
scores = []
from pythonlib.tools.plottools import annotate, colorGradient

# 1) Get list of all scores
for i, p in enumerate(probedat):
    scores.append({
        "modelcomp_offline":p["modelcomp_offline"],
    "trial":i})
    for name, PD in PDdict.items():
        scores[-1][name] = PD.Datamodel.trials[i]["posterior"]

# 2) for each trials assign bin based on percentile
klist = scores[0].keys()
N = 10
scores_binned = {}
for k in klist:
    vals = [s[k] for s in scores]
    if k=="trial":
        scores_binned[k] = vals
    else:
        bins = np.percentile(vals, np.linspace(0, 100, N))
#         bins = np.linspace(min(vals), max(vals), N)
        vals_binned = np.digitize(vals, bins)

        knew = f"{k}_binned"
        scores_binned[knew] = vals_binned

# 3) Make gridplot
xfeat = "3line_binned"
yfeat = "linePlusL_binned"

xvals = sorted(list(set(scores_binned[xfeat])))
yvals = sorted(list(set(scores_binned[yfeat])))

def f1(t, ax):
    PD = PDdict["3line"] # doesnt matter which, since behavior isa ll same.
    PD.Datamodel.plotTrialStrokes(t, ax, ver="beh")
def f2(t, ax):
    PDdict["3line"].Datamodel.plotTrialStrokes(t, ax, ver="task_maxlikeli")
def f3(t, ax):
    PDdict["linePlusL"].Datamodel.plotTrialStrokes(t, ax, ver="task_maxlikeli")
plotfunlist = [f1, f2, f3]

for n in range(10): # diff examples
    figlist =[]
    axeslist =[]
    for _ in range(len(plotfunlist)):
        fig, axes = plt.subplots(ncols=len(xvals), nrows=len(yvals), sharex=True, sharey=True, 
                                 figsize=(3*len(xvals), 3*len(yvals)))
        figlist.append(fig)
        axeslist.append(axes)

    for i, x in enumerate(xvals):
        for j, y in enumerate(yvals):

            # pick a random trials that matches these bins
            idx = np.logical_and(scores_binned[xfeat]==x, scores_binned[yfeat]==y)
            idx = np.where(idx)[0]
            trials = [scores_binned["trial"][i] for i in idx]

            if len(trials)>0:
                t = random.sample(trials,1)[0]

                for plotfun, axes in zip(plotfunlist, axeslist):

                    ax = axes[j][i]
                    plotfun(t, ax)

                    # -- label the edge panels
                    if j==len(yvals)-1:
                        ax.set_xlabel(f"{xfeat}:{x}")
                    if i==0:
                        ax.set_ylabel(f"{yfeat}:{y}")

                    mc = [s["modelcomp_offline"] for s in scores if s["trial"]==t][0]
                    m1 = [s["3line"] for s in scores if s["trial"]==t][0]
                    m2 = [s["linePlusL"] for s in scores if s["trial"]==t][0]

                    # -- annotate with model scores
                    col = colorGradient((0.5+mc/2), col1=[0,0,1], col2=[1,0,0])
                    ax.set_title(f"sc{mc:.2f}|3l{m1:.2f}|lL{m2:.2f}", color=col)


    # save 
    for i, f in enumerate(figlist):
        f.savefig(f"{SAVEDIR}/overviewgrid-{expt}-3lineVslinePlusL-run{n}-{i}.pdf")    
    plt.close("all")



In [ ]:
figlist[0].savefig([

In [ ]:
[i for i in idx]

In [ ]:
np.logical_and(scores_binned["modelcomp_offline_binned"]==6, scores_binned[yfeat]==8)

In [ ]:
scores_binned["modelcomp_offline_binned"]==6

In [ ]:
sc

In [ ]:
np.where((scores_binned[xfeat]==x) & (scores_binned[yfeat]==y))

In [ ]:
np.where((scores_binned[xfeat]==x) & (scores_binned[yfeat]==y))

In [ ]:
x

In [ ]:
pd.DataFrame(scores_binned)

In [ ]:
## PLOT EXAMPLE TRAILS ACROSS DISTRIBUTION OF SCORES

# for each trial plot behavior + best parse for all models



## [BELOW] trying out

In [ ]:
def prepDat(fd, trials):
#     trials = [t for t in trials if getTrialsFixationSuccess(fd, t)]
    strokes, tasks, fix, strokes_task = [[], [], [], []]
    for t in trials:
        S = getTrialsStrokesByPeanuts(fd, t)
        if len(S)>0:
#             trials = [t for t in trials if len(getTrialsStrokesByPeanuts(fd, t))>0]
            strokes.append(S)
            tasks.append(getTrialsTask(fd, t))
            fix.append(getTrialsFix(fd, t)["fixpos_pixels"])
            strokes_task.append(getTrialsTaskAsStrokes(fd, t))
    for s, t, f in zip(strokes_task, tasks, fix):
        t["strokes"] = s
        t["fixpos"] = f
    return strokes, tasks


In [ ]:
from pythonlib.drawmodel.analysis import *

# 1) Prepare data
trials = getIndsTrials(fd)
trials = [t for t in trials if not getTrialsTaskProbeKind(fd, t)=="train"]
strokes, tasks = prepDat(fd, trials)

In [ ]:
# # 2) Input dataset
# data = Dataset(strokes, tasks)

# # 3) Prepare model
# PRIORVER = "uniform" # CAHNGE THIS
# LIKELIVER = "segments"
# MODELNAME = "test"
# priorFunction, NORM_VER = makePriorFunction(ver=PRIORVER)
# likeliFunction = makeLikeliFunction(ver=LIKELIVER)
# mod = Model(MODELNAME, priorFunction, likeliFunction)

# # 4) apply model to score all beahvuior
# POSTERIOR_VER = "weighted"
# data.applyModel(mod, prior_ver= NORM_VER, posterior_ver=POSTERIOR_VER)

# # 5) Plot posterior scores
# data.plotPosteriorHist()
# # plt.title(title)

In [ ]:

# task = data1.trials[0]["task"]
# parsefun(task)

# from tools.tasks import task2parses
# P = task2parses(task, parsefun)
# len(P)


In [ ]:
## ==== KEY: comparison between two models.

def fit(modelstr):
    # Input dataset
    data = Dataset(strokes, tasks)

    # 3) Prepare model
    PRIORVER = "uniform"
    LIKELIVER = "segments"
    MODELNAME = modelstr
    priorFunction, NORM_VER = makePriorFunction(ver=PRIORVER)
    likeliFunction = makeLikeliFunction(ver=LIKELIVER)
    mod = Model(MODELNAME, priorFunction, likeliFunction, parse_ver="chunks", chunkmodel = modelstr)

    # 4) apply model to score all beahvuior
    POSTERIOR_VER = "maxlikeli"
    data.applyModel(mod, prior_ver= NORM_VER, posterior_ver=POSTERIOR_VER, parses_bothdir=False)

    if False:
        # 5) Plot posterior scores
        data.plotPosteriorHist()
        # plt.title(title)
        
    return data


# 1) get two dataset instaences (same data, diff model)
# data1 = fit("linePlusL")
data1 = fit(parsefun)
data2 = fit("3line")

In [ ]:
# === NULL MODELS

# 1) MINIMIZE TOTAL TRAVELED DISTANCE
datanull = Dataset(strokes, tasks)

# 3) Prepare model
PRIORVER = "prox_to_origin"
LIKELIVER = "segments"
MODELNAME = PRIORVER
priorFunction, NORM_VER = makePriorFunction(ver=PRIORVER)
likeliFunction = makeLikeliFunction(ver=LIKELIVER)
mod = Model(MODELNAME, priorFunction, likeliFunction, parse_ver="permutations")

# 4) apply model to score all beahvuior
POSTERIOR_VER = "weighted"
datanull.applyModel(mod, prior_ver= NORM_VER, posterior_ver=POSTERIOR_VER, parses_bothdir=False)



In [ ]:
data1.plotPosteriorHist()
data2.plotPosteriorHist()
datanull.plotPosteriorHist()

In [ ]:
from scipy.special import softmax

x = np.array([1, 5, 3, 2])
x = x-100
beta = 0.01
x = beta * x
softmax(x)

In [ ]:
# 2) for each trial, compare scores.
t = random.randrange(0, len(tasks))

data1.plotExampleTrial(t, sort_by_likeli=True)
data2.plotExampleTrial(t, sort_by_likeli=True)
datanull.plotExampleTrial(t, sort_by_likeli=True)

## Fit model to training data




In [ ]:
PARAMS = {
    "getPriors":{"normscore":[0.001]},
    "getLikelis":None,
    "getPosteriors":None,
    "parse":{"parses_split":False, "parses_bothdir":False},
    "posterior_ver":"weighted",
    "prior_norm_ver":"softmax",
    "standardize":{"standardize_strokes":False},
    "run_only_once":['parse', 'prior_score', 'getLikelis']
}

# 3) Prepare model
PARAMS_MODEL = {
    "priorver":"uniform",
    "likeliver":"segments",
    "parse_ver":"permutations"
}
PARAMS_MODEL["modelname"]="test"
def getmodel(PARAMS_MODEL):
    priorFunction, NORM_VER = makePriorFunction(ver=PARAMS_MODEL["priorver"])
    likeliFunction = makeLikeliFunction(ver=PARAMS_MODEL["likeliver"])
    mod = Model(PARAMS_MODEL["modelname"], priorFunction, likeliFunction,
                parse_ver=PARAMS_MODEL["parse_ver"])
    return mod

# 4) apply model to score all beahvuior
datanull = Dataset(strokes, tasks, PARAMS=PARAMS)
datanull.applyModel(getmodel(PARAMS_MODEL))


In [ ]:
out =[]
for x in np.linspace(-5, 0, 10):
    beta = 1*10**x
    datanull.PARAMS["getPriors"]["normscore"] = [beta]
    datanull.run()
    posteriors = [t["posterior"] for t in datanull.trials]
    
    for post in posteriors:
        out.append({
            "beta":beta,
            "posts":post})
                        
                        


In [ ]:
import pandas as pd
import seaborn as sns

DF = pd.DataFrame(out)
sns.catplot(data=DF, x="beta", y="posts", aspect=4)
sns.catplot(data=DF, x="beta", y="posts", aspect=4, kind="point")

## train and test - cross validation



### optimization code

In [ ]:
datanull.trials[0]["posterior"]

In [ ]:
t = random.randrange(0, len(tasks))
datanull.plotExampleTrial(t, sort_by_likeli=True)

In [ ]:
data1.plotExampleTrial(t, sort_by_likeli=True)

In [ ]:
## ==


In [ ]:
data2.plotPosteriorHist()

In [ ]:
strokes_beh = data2.trials[t]["behavior"]["strokes"]
strokes_task = data2.trials[t]["model_parses"][0]["strokes"]

In [ ]:
fig, axes = plt.subplots(ncols=2)
plotDatStrokes(strokes_beh, ax=axes[0])
plotDatStrokes(strokes_task, ax=axes[1])

In [ ]:
distanceDTW(strokes_beh, strokes_task, ver="segments", asymmetric=False)

In [ ]:
distanceDTW(strokes_beh, strokes_task, ver="segments", asymmetric=True)

# NOTES:
Softmax is hacky right now.

In [ ]:
def _run(prior_ver, likeli_ver="split_segments", posterior_ver="weighted", title=""):
    dset = makeDataset(filedata, trials_list)
    priorFunction, NORM_VER = makePriorFunction(ver=prior_ver)
    likeliFunction = makeLikeliFunction(ver=likeli_ver)
    mod = Model(modelname, priorFunction, likeliFunction)
    dset.applyModel(mod, prior_ver= NORM_VER, posterior_ver=posterior_ver)
    dset.plotPosteriorHist()
    plt.title(title)
    return dset

# 1) random model
_run("uniform", "split_segments", "weighted", "uniform")

# 2) positive control model [max likelis]
_run("uniform", "split_segments", "maxlikeli", "maxlikeli")


# 3) positive control model [weigh likelis by softmax(likelis)
_run("uniform", "split_segments", "likeli_weighted", "likeli_weighted")


# 4) Hypothesis: distance finger travel
dset = _run("distance_travel", title="distance_travel")

# 5) Hypothesis, angle bias
dset = _run("angle_test", title="angle_test")

dset.plotExampleTrial(3)

# 5) Hypothesis, angle bias
dset = _run("angle_test", title="angle_test")

In [ ]:
from pythonlib.drawmodel.analysis import *

trials = range(1,100)

strokes = [getTrialsStrokesByPeanuts(fd, t) for t in trials if getTrialsFixationSuccess(fd,t)]

tasks = [getTrialsTask(fd, t) for t in trials if getTrialsFixationSuccess(fd,t)]
fix = [getTrialsFix(fd, t)["fixpos_pixels"] for t in trials if getTrialsFixationSuccess(fd,t)]
strokes_task = [getTrialsTaskAsStrokes(fd, t) for t in trials if getTrialsFixationSuccess(fd,t)]
for s, t, f in zip(strokes_task, tasks, fix):
    t["strokes"] = s
    t["fixpos"] = f
    
dset = Dataset(strokes, tasks)

# === APPLY MODEL
PRIORVER = "uniform" # CAHNGE THIS
LIKELIVER = "segments"
MODELNAME = "test"
priorFunction, NORM_VER = makePriorFunction(ver=PRIORVER)
likeliFunction = makeLikeliFunction(ver=LIKELIVER)
mod = Model(MODELNAME, priorFunction, likeliFunction)

# === apply model to score all beahvuior
POSTERIOR_VER = "maxlikeli"
dset.applyModel(mod, prior_ver= NORM_VER, posterior_ver=POSTERIOR_VER)
dset.plotPosteriorHist()
plt.title(title)


In [ ]:
# == make prior function that allows only things chunkable.
# so is uniform, based on chunks


In [ ]:
dset.model.parses_both_direction

In [ ]:
# DEVELOPMENT - MAIN ANALYSES

## BELOW - OLD SCRATCH

In [ ]:
# === LOAD ALL SESSIONS FOR A DAY
# A = [("lines1", "Red", 200831),
#  ("lines1", "Pancho", 200831),
#  ("ESC1", "Red", 200830),
#  ("ESC1", "Pancho", 200830)]
A = [("lines2", "Red", 200907),
 ("lines2", "Pancho", 200907)]

for (expt, animal, date) in A:

    # expt = "lines1"
    # animal = "Red"
    # date = 200831

    # expt = "ESC1"
    # animal = "Red"
    # date = 200830

    # simple, just try many sessions...
    fdsessions = []
    FD = {}
    N = 12
    for session in range(10):
        fd = loadSingleData(animal, date, expt, session, resave_as_dict=False, load_resaved_data=True, 
                          resave_overwrite=False)
        if fd is not None:
            print(f"appending fd for sess {session}")
            fdsessions.append({
                "session": session,
                "fd":fd})
            FD[session] = fd
            if session==N-1:
                assert False, "got the end and still found.. you need to search for even more sessions..."

            SAVEDIR = f"{fd['params']['figuredir_notebook']}/probes/{animal}_{date}"
            import os
            os.makedirs(SAVEDIR, exist_ok=True)


    print("--- SUMMARY")
    for f in fdsessions:
        print(f"session: {f['session']}, ntrials: {f['fd']['params']['n_trials']}")


    # === categorize all trials based on task/probe, etc
    probedat = []
    ct = 0
    for sess, fd in FD.items():
        for t in getIndsTrials(fd):
            if getTrialsFixationSuccess(fd, t):
                probe = getTrialsTaskProbeInfo(fd, t)
                task = getTrialsTask(fd, t)
                
#                 print(probe)
                
                if probe["prototype"]==0 and len(probe["saved_setnum"])==0 and probe["resynthesized"]==0:
                    # then this is new random task sampled each block.
                    randomtask = True
                else:
                    randomtask = False

                if probe["probe"]==0:
                    kind = "train"
                else:
                    if probe["feedback_ver"]=="same_as_task" and randomtask==False:
                        if probe["constraints_to_skip"]=={}:
                            kind = "probe1_liketrain"
                        elif "strokes" in probe["constraints_to_skip"].values():
                            kind = "probe1_nostrokeconstraint"
                    elif probe["feedback_ver"]=="same_as_task" and randomtask==True and probe["constraints_to_skip"]=={}:
                        kind = "probe2_liketrain"
                    elif probe["feedback_ver"]=="same_as_task" and randomtask==True and "strokes" in probe["constraints_to_skip"].values():
                        kind = "probe2_nostrokeconstraint"
                    elif probe["feedback_ver"] in ["thresh_active", "mid_reward"] and randomtask==False and "strokes" in probe["constraints_to_skip"].values():
                        kind = "probe3"
                    elif probe["feedback_ver"] in ["same_except_model"] and randomtask==False and "strokes" in probe["constraints_to_skip"].values():
                        kind = "probe3_hdpos"
                    elif probe["feedback_ver"] in ["thresh_active", "mid_reward"] and randomtask==True and "strokes" in probe["constraints_to_skip"].values():
                        kind = "probe4"
                    else:
                        print(probe)
                        assert False, "what kind is this?"

                probedat.append({
                    "session":sess,
                    "trial":t,
                    "trial_day":ct,
                    "kind":kind,
                    "taskname": probe["unique_name"],
                    "stage":task["stage"],
                    "block":getTrialsBlock(FD[sess], t),
                    "prototype":probe["prototype"],
                    "constraints_to_skip":probe["constraints_to_skip"]})
                ct+=1


    # == for each category, list names of all tasks
    kindlist = set([p["kind"] for p in probedat])
    print("probe kinds -- trials");
    for kind in kindlist:
        print(" ")
        print(kind)
        trials = [p["trial"] for p in probedat if p["kind"]==kind]
        tasknames = [p["taskname"] for p in probedat if p["kind"]==kind]
    #     print(trials)
        print(set(tasknames))
        plt.figure()
        plt.plot(trials, tasknames)

    import seaborn as sns
    import pandas as pd
    dframe = pd.DataFrame(probedat)

    # sns.scatterplot(x="trial", y="taskname", data=dframe, hue="kind")
    fig, axes = plt.subplots(5, 1, sharex=True, squeeze=False, figsize=(28, 10))

    sns.scatterplot(x="trial_day", y="kind", data=dframe, hue="kind", ax=axes[0][0])
    sns.scatterplot(x="trial_day", y="stage", data=dframe, hue="kind", ax=axes[1][0])
    sns.scatterplot(x="trial_day", y="stage", data=dframe, hue="prototype", ax=axes[2][0])
    sns.lineplot(x="trial_day", y="session", data=dframe, ax=axes[3][0])
    sns.lineplot(x="trial_day", y="block", data=dframe, ax=axes[4][0])

    fig.savefig(f"{SAVEDIR}/overviewAllTrials.pdf")



    stagelist = set([p["stage"] for p in probedat])
    sesslist = set([p["session"] for p in probedat])    
    kindlist = set([p["kind"] for p in probedat])

    ## PLOT PROBE TASKS OVER THE ENTIRE DAY
    maxtrialsplot = 80

    for kind in kindlist:
        for stage in stagelist:
            for s in sesslist:
                trialsthis = [p["trial"] for p in probedat if p["session"]==s and p["kind"]==kind and p["stage"]==stage]
                fd = FD[s]
                print(f"-- for session {s}, kind {kind}, stage {stage}; trials:")
                print(trialsthis)

                if len(trialsthis)>0:
                    if len(trialsthis)>maxtrialsplot:
                        trialsthis = sorted(random.sample(trialsthis, maxtrialsplot))                        
                        r = None
                    else:
                        r = None
                    titles = []
                    for t in trialsthis:
                        mscore = getTrialsBehEvaluation(fd, t)["output"]["modelscore"]["value"][0][0]
                        bscore = getTrialsBehEvaluation(fd, t)["beh_multiplier"][0][0]                                               
                        titles.append(f"{t},bk{getTrialsBlock(fd, t)},{getTrialsTask(fd, t)['str']}\nmsco{mscore:.2f},bsco{bscore:.2f}")

#                         titles = [f"{t},bk{getTrialsBlock(fd, t)}{getTrialsTask(fd, t)['str']}\nmscore{mscore}" for t in trialsthis]
                    # plot
                    
                    # construct titles
                    
                    fig1 = plotMultTrialsSimple(fd, trialsthis, zoom=True, strokes_ver="peanuts", plot_fix=True,
                                            plotver="strokes", rand_subset = r, titles=titles)

                    scores = [getTrialsScoreRecomputed(fd, t, normalize=True) for t in trialsthis]
                    scores_compos = [getTrialsScoreRecomputed(fd, t, ver="DTW_min", normalize=True) for t in trialsthis]
                    scores_compos2 = [getTrialsScoreRecomputed(fd, t, ver="DTW_min_minus_max", normalize=True) for t in trialsthis]

                    # == plot quick scores
                    fig2 = plt.figure(figsize=(10, 12))

                    plt.subplot(311)
                    plt.title(f"session {s}, kind {kind}, stage {stage}")
                    plt.plot(trialsthis, scores, "ok", label="pts")
    #                 plt.plot(trialsthis, scores_compos, "or", label="compositonal")
    #                 plt.plot(trialsthis, scores_compos2, "og", label="compositonal_min_minus_max")
                    plt.ylabel("score (pts) (norm HD, high is good)")
                    plt.ylim([-0.2, 1])
                    plt.legend()

                    plt.subplot(312)
                    plt.title(f"session {s}, kind {kind}, stage {stage}")
    #                 plt.plot(trialsthis, scores, "ok", label="pts")
                    plt.plot(trialsthis, scores_compos, "or", label="compositonal")
    #                 plt.plot(trialsthis, scores_compos2, "og", label="compositonal_min_minus_max")
                    plt.ylabel("score (compositonal) (norm HD, high is good)")
                    plt.ylim([-0.5, 1])
                    plt.legend()

                    plt.subplot(313)
                    plt.title(f"session {s}, kind {kind}, stage {stage}")
    #                 plt.plot(trialsthis, scores, "ok", label="pts")
    #                 plt.plot(trialsthis, scores_compos, "or", label="compositonal")
                    plt.plot(trialsthis, scores_compos2, "og", label="compositonal_min_minus_max")
                    plt.ylabel("score (compositonal_min_minus_max) (norm HD, high is good)")
                    #     plt.ylim([-0.2, 1])
                    plt.legend()

    #                 plt.subplot(212)
    #                 s1 = (scores - np.mean(scores))/np.std(scores)
    #                 s2 = (scores_compos - np.mean(scores_compos))/np.std(scores_compos)
    #                 s3 = (scores_compos2 - np.mean(scores_compos2))/np.std(scores_compos2)
    #                 plt.plot(trialsthis, s1, "ok", label="pts")
    #                 plt.plot(trialsthis, s2, "or", label="compositonal")
    #                 plt.plot(trialsthis, s3, "og", label="compositonal_min_minus_max")
    #                 plt.title("z-scored")
    #                 plt.ylabel("score (recomputed) (norm HD, high is good)")
    #                 plt.ylim([-3, 3])
    #                 plt.legend()

                    # save
                    fig1.savefig(f"{SAVEDIR}/trialsByProbeKind_{kind}-{stage}-sess{s}-fig1.pdf")
                    fig2.savefig(f"{SAVEDIR}/trialsByProbeKind_{kind}-{stage}-sess{s}-fig2.pdf")
    #                 fig2.savefig(f"{SAVEDIR}/trialsByProbeKind_sess{s}-{kind}-{stage}-fig2.pdf")





In [ ]:
getTrialsTask(fd, t)["savedTaskSet"]

In [ ]:
expt = "lines2"
animal = "Pancho"
date = 200904

fdsessions = []
FD = {}
N = 12
for session in range(10):
    fd = loadSingleData(animal, date, expt, session, resave_as_dict=False, load_resaved_data=True, 
                      resave_overwrite=False)
    if fd is not None:
        print(f"appending fd for sess {session}")
        fdsessions.append({
            "session": session,
            "fd":fd})
        FD[session] = fd
        if session==N-1:
            assert False, "got the end and still found.. you need to search for even more sessions..."

        SAVEDIR = f"{fd['params']['figuredir_notebook']}/probes/{animal}_{date}"
        import os
        os.makedirs(SAVEDIR, exist_ok=True)


print("--- SUMMARY")
for f in fdsessions:
    print(f"session: {f['session']}, ntrials: {f['fd']['params']['n_trials']}")


# === categorize all trials based on task/probe, etc
probedat = []
ct = 0
for sess, fd in FD.items():
    for t in getIndsTrials(fd):
        if getTrialsFixationSuccess(fd, t):
            probe = getTrialsTaskProbeInfo(fd, t)
            task = getTrialsTask(fd, t)

            if probe["probe"]==0:
                kind = "train"
            else:
                if probe["feedback_ver"]=="same_as_task" and probe["prototype"]==1:
                    if probe["constraints_to_skip"]=={}:
                        kind = "probe1_liketrain"
                    elif "strokes" in probe["constraints_to_skip"].values():
                        kind = "probe1_nostrokeconstraint"
                elif probe["feedback_ver"]=="same_as_task" and probe["prototype"]==0 and probe["constraints_to_skip"]=={}:
                    kind = "probe2_liketrain"
                elif probe["feedback_ver"]=="same_as_task" and probe["prototype"]==0 and "strokes" in probe["constraints_to_skip"].values():
                    kind = "probe2_nostrokeconstraint"
                elif probe["feedback_ver"] in ["thresh_active", "mid_reward"] and probe["prototype"]==1 and "strokes" in probe["constraints_to_skip"].values():
                    kind = "probe3"
                elif probe["feedback_ver"] in ["same_except_model"] and probe["prototype"]==1 and "strokes" in probe["constraints_to_skip"].values():
                    kind = "probe3_hdpos"
                elif probe["feedback_ver"] in ["thresh_active", "mid_reward"] and probe["prototype"]==0 and "strokes" in probe["constraints_to_skip"].values():
                    kind = "probe4"
                else:
                    print(probe)
                    assert False, "what kind is this?"

            probedat.append({
                "session":sess,
                "trial":t,
                "trial_day":ct,
                "kind":kind,
                "taskname": probe["unique_name"],
                "stage":task["stage"],
                "prototype":probe["prototype"],
                "constraints_to_skip":probe["constraints_to_skip"]})
            ct+=1

In [ ]:

## for each trial also get online and offline score
for p in probedat:
    s = p["session"]
    t = p["trial"]
    
    p["bk"] = getTrialsBlock(FD[s], t)
    p["bq"] = getTrialsBloque(FD[s], t)
    
    # score
    if getTrialsFixationSuccess(FD[s], t):
        p["score_model"] = getTrialsOutcomesWrapper(FD[s], t)["beh_evaluation"]["output"]["modelscore"]["value"][0][0]
        p["score_hd"] = getTrialsOutcomesWrapper(FD[s], t)["beh_evaluation"]["output"]["hausdorff"]["value"][0][0]
        p["score_offline"] = getTrialsScoreRecomputed(FD[s], t)
    
    
    
    

import pandas as pd

dfthis = pd.DataFrame(probedat)
dfthis

In [ ]:
### import seaborn as sns

y = "score_model"
sns.relplot(x = "trial", y=y, hue="bk", data=dfthis, row="kind", kind="scatter", aspect=3)
sns.relplot(x = "trial_day", y=y, hue="bk", data=dfthis, row="kind", kind="scatter", aspect=3)
sns.relplot(x = "bq", y=y, hue="bk", data=dfthis, row="kind", kind="scatter", aspect=3)

In [ ]:
### import seaborn as sns

y = "score_model"
sns.catplot(x = "bk", y=y, data=dfthis, row="kind", kind="point", aspect=3)
# sns.catplot(x = "bq", y=y, data=dfthis, col="kind", row="bk", kind="point", aspect=3)
sns.catplot(x = "bq", y=y, data=dfthis, row="kind", hue="bk", kind="point", aspect=3)

In [ ]:
## COMPUTE COMPOSITIONAL SCORE
# [Quick version] DTW score relative to non-DTW score

## ==== [testing] stroke based hd
from pythonlib.tools.stroketools import distanceDTW
t = random.sample(getIndsTrials(fd),1)[0]

strokes_beh = getTrialsStrokesByPeanuts(fd, t, replaynum=1)
strokes_task = getTrialsTaskAsStrokes(fd, t)

for ass in [True, False]:
    print(f"assymetric: {ass}")
    plotTrialSimple(fd, t, zoom=True, plot_fix=False, plotver="strokes", 
                    use_peanut_params={'replaynum': 1, 'active': True})

    print(distanceDTW(strokes_beh, strokes_task[::-1], ver="segments", asymmetric=ass))
    print(distanceDTW(strokes_beh, strokes_task, ver="segments", asymmetric=ass))

    plt.figure(figsize=(10,10))
    ax = plt.subplot(211)
    plotDatStrokes(strokes_beh, ax=ax)
    plotDatStrokes(strokes_task[::-1], ax=ax)

    ax = plt.subplot(212)
    plotDatStrokes(strokes_beh, ax=ax)
    plotDatStrokes(strokes_task, ax=ax)

# compute minimum score for all permutations of task strokes
# make assymetric false, so forced to use all task strokes.
print("-- all permutations")
from itertools import permutations
scores =[]
for s in permutations(strokes_task):
    print(distanceDTW(strokes_beh, s, ver="segments", asymmetric=False))
    scores.append(distanceDTW(strokes_beh, s, ver="segments", asymmetric=False)[0])
score = min(scores)
print(score)

In [ ]:
from pythonlib.tools.listtools import permuteRand

permuteRand([1,2,3], 6, not_enough_ok=True)
from itertools import permutations

for i in permutations([1,2,3]):
    print(i)


## SCRATCH

In [ ]:
featurestoplot = []
for key, val in getTrialsBlockParams(fd, 1)["behEval"]["beh_eval"].items():
    if val["feature"] =="hausdorff" and val["weight"][0][0]>0:
        featurestoplot.append("hausdorff")
    if val["feature"] =="frac_touched" and val["weight"][0][0]>0:
        featurestoplot.append("frac_touched")
featurestoplot.append("score_offline")

fig1, fig2 = plotOverview_(df, featurestoplot=featurestoplot)
fig1.savefig(f"{SAVEDIRDAY}/overview1.pdf")
fig2.savefig(f"{SAVEDIRDAY}/overview2.pdf")

# 2) relationship between reward and factors that go into reward
figs = plotReward(df, featurestoplot=featurestoplot)
for i, f in enumerate(figs):
    f.savefig(f"{SAVEDIRDAY}/reward_score_{i}.pdf")

# 3) PLOT BEHAVIOR FOR TRIALS SORTED BY SCORE
import copy
scoretypes = copy.copy(featurestoplot)
scoretypes.extend(["behscore", "reward"])
for score_type in scoretypes:
    FIGS = plotBehSortedByScore(df, fd, score_type)
    for ver, figs in FIGS.items():
        for i, f in enumerate(figs):
            f.savefig(f"{SAVEDIRDAY}/trialsSortedByScore_{score_type}_{ver}_{i}_.pdf")

# 4) Plot behavior subsampling in chronological order
trials = [t for t in getIndsTrials(fd) if getTrialsFixationSuccess(fd, t)]
Nrand = 80
fig = plotMultTrialsSimple(fd, trials, zoom=True, strokes_ver="peanuts", plot_fix=False,
                        plotver="strokes", rand_subset=Nrand)
fig.savefig(f"{SAVEDIRDAY}/trialsRandomChronOrder.pdf")

# 5) TASK VISUALIZATIONS, SCHEDULE, REPETITION
figs = plotTaskSchedules(df)
for i, f in enumerate(figs):
    f.savefig(f"{SAVEDIRDAY}/taskSchedule{i}.pdf")

In [ ]:
if False:
    animal = "Pancho"
    expt = "pilot"
    date = 200822
    session = 1

    fd = loadSingleData(animal, date, expt, session, resave_as_dict=False, load_resaved_data=True, 
                              resave_overwrite=False)

    ## ==== [testing] stroke based hd
    from pythonlib.tools.stroketools import distanceDTW
    t = random.sample(getIndsTrials(fd),1)[0]

    for ass in [True, False]:
        print(f"assymetric: {ass}")
        plotTrialSimple(fd, t, zoom=True, plot_fix=False, plotver="strokes", 
                        use_peanut_params={'replaynum': 1, 'active': True})

        strokes_beh = getTrialsStrokesByPeanuts(fd, t, replaynum=1)
        strokes_task = getTrialsTaskAsStrokes(fd, t)
        print(distanceDTW(strokes_beh, strokes_task[::-1], ver="segments", asymmetric=ass))
        print(distanceDTW(strokes_beh, strokes_task, ver="segments", asymmetric=ass))

        plt.figure(figsize=(10,10))
        ax = plt.subplot(211)
        plotDatStrokes(strokes_beh, ax=ax)
        plotDatStrokes(strokes_task[::-1], ax=ax)

        ax = plt.subplot(212)
        plotDatStrokes(strokes_beh, ax=ax)
        plotDatStrokes(strokes_task, ax=ax)


In [ ]:
## ==== plot scoring separated by task type

df = extractSessionDf(fd)

# === add note about what probe type this is



## TO DO:
(1) get all permutations
(2) normalize by num strokes.
(3) systematic - compare to old version.